In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

In [ ]:
titanic = pd.read_csv('titanic.csv', sep=';', index_col='PassengerId')

# Идиома: расчет частоты события

- `.sum()` вектора из нулей и единиц = количество единиц
- `.mean()` вектора из нулей и единиц = доля единиц

In [ ]:
titanic.Survived.mean()

Какова частота выполнения условия?

In [ ]:
(titanic.Age >= 18).mean()

Сколько всего строк, удовлетворяющих условию?

In [ ]:
(titanic.Age >= 18).sum()

# GroupBy

Как посчитать долю выживших в разных группах?

In [ ]:
print 'female\t', titanic.loc[titanic.Sex == 'female', 'Survived'].mean()
print 'male\t', titanic.loc[titanic.Sex == 'male', 'Survived'].mean()

In [ ]:
titanic.Parch.unique()

[Split -> Apply -> Combine](http://pandas.pydata.org/pandas-docs/stable/groupby.html)

Аналогично `GROUP BY` в SQL:
```sql
SELECT Sex, AVG(Survived)
FROM titanic
GROUP BY Sex;
```

In [ ]:
titanic.groupby('Sex').Survived.mean()

In [ ]:
titanic.groupby('Parch').Survived.mean()

## 1) Разбиение (Split)

GroupBy хранит исходный `DataFrame` (или `Series`) и разбиение на группы, т.е. соответствие "название группы (значение колонки, по которой группируем) – список индексов"

In [ ]:
splits = titanic.groupby('Sex')
print type(splits)

In [ ]:
splits.groups

In [ ]:
splits.get_group('female')  # тот же результат, что и titanic.loc[titanic.Sex == 'female']

Другие способы разбиения:

- по нескольким колонкам

In [ ]:
titanic.groupby(['Sex', 'Pclass']).groups.keys()

- по колонке, которой нет в исходном `DataFrame`

In [ ]:
adult = (titanic.Age >= 18)
adult.loc[titanic.Age.isnull()] = np.nan

In [ ]:
adult.value_counts(dropna=False)

In [ ]:
titanic.groupby(adult).groups.keys()

NB: группа, соответствующая значению `NaN`, не создается. Можно это обойти, предварительно заполнив `NaN` каким-нибудь специальным значением.

In [ ]:
titanic.groupby(adult.fillna('NA')).groups.keys()

- по значению функции, рассчитанной по индексу

In [ ]:
titanic.groupby(lambda s: s % 3).groups

In [ ]:
titanic.groupby(titanic.index % 3).groups  # то же самое

NB: можно разбивать не только по строкам, но и по столбцам (`axis=1`)

## 2) Агрегация (Apply + Combine)

Обычные функции агрегации:

In [ ]:
splits.count()

In [ ]:
splits.nunique()

In [ ]:
splits.mean()

Доступ к отдельному столбцу с тем же самым разбиением:

In [ ]:
splits.Survived

In [ ]:
splits.Survived.mean()  # быстрее, чем spilts.mean().Survived

In [ ]:
splits.Cabin.nunique()

Несколько функций сразу:

In [ ]:
splits.Survived.agg(['mean', 'std', 'count'])

Первая/последняя/n-ая запись внутри каждой группы:

In [ ]:
splits.first()

In [ ]:
splits.last()

In [ ]:
splits.nth(0)

Произвольная функция:

In [ ]:
def oldest(df):
    return df.loc[df.Age.argmax()]

In [ ]:
splits.apply(oldest)

# Работа со строками

[Документация](https://pandas.pydata.org/pandas-docs/stable/text.html)

In [ ]:
titanic.Name.str

In [ ]:
titanic.Name.apply(lambda s: s.split(',')[0])

In [ ]:
titanic.Name.str.split(',').str[0]

In [ ]:
titanic.Name.str.split(', ', n=1, expand=True)

In [ ]:
titanic.Name.str.len()

In [ ]:
titanic.Name.str.upper()

In [ ]:
titanic.Name.str.contains('Mr.')

In [ ]:
titanic.Ticket.str.isdigit()

# Простая визуализация

[Официальная документация с примерами](https://pandas.pydata.org/pandas-docs/stable/visualization.html)

`pandas` рисует графики с помощью библиотеки `matplotlib`

In [ ]:
# Заклинание, нужное для отрисовки графиков непосредственно в ноутбуке (подробнее в лекции 7)
%matplotlib inline
# Настройка стиля и размера графиков
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 6.0)
matplotlib.pyplot.style.use('ggplot')

In [ ]:
titanic.plot()

In [ ]:
titanic.Age.plot.hist(bins=20)

In [ ]:
titanic.plot.scatter('Fare', 'Age')

In [ ]:
titanic.Parch.value_counts(sort=False).plot.bar()

# Аналитика рынка акций

## Один файл, один лист

In [ ]:
df = pd.read_excel('./stocks_data/2016-11-23.xls', sheetname='AAPL')

In [ ]:
df

In [ ]:
df.dtypes

Преобразуем поле `Time` к типу `datetime`:

In [ ]:
df.Time = pd.to_datetime('2016-11-23 ' + df.Time)

In [ ]:
df.set_index('Time', inplace=True)

In [ ]:
df[['Open', 'High', 'Low', 'Close']].plot()

## Один файл, все листы

In [ ]:
dfs = pd.read_excel('./stocks_data/2016-11-23.xls', sheetname=None)

In [ ]:
type(dfs)

In [ ]:
dfs.keys()

In [ ]:
dfs['AAPL']

In [ ]:
dfs['GOOG']

Объединение таблиц:

In [ ]:
pd.concat([dfs['AAPL'], dfs['GOOG']])

In [ ]:
pd.concat([dfs['AAPL'], dfs['GOOG']], ignore_index=True)

In [ ]:
pd.concat([dfs['AAPL'], dfs['GOOG']], axis=1)

In [ ]:
pd.concat([dfs['AAPL'].add_prefix('AAPL_'), dfs['GOOG'].add_prefix('GOOG_')], axis=1)

`pd.merge()` – продвинутый способ объединения таблиц. Важные параметры:
- `how='inner'` – тип джойна: `'inner'` (пересечение индексов),`'outer'` (объединение индексов), `'left'`/`'right'` (индексы левой/правой таблицы)
- `left_on='col_name'` или `left_index=True` – поле (или индекс) для джойна левой таблицы
- `right_on='col_name` или `right_index=True` – поле (или индекс) для джойна правой таблицы

In [ ]:
pd.merge(dfs['AAPL'], dfs['GOOG'], how='outer', on='Time', suffixes=('_AAPL', '_GOOG'))

In [ ]:
pd.concat([dfs['AAPL'], dfs['GOOG']])

In [ ]:
for ticker, df in dfs.iteritems():
    df['Ticker'] = ticker

In [ ]:
dfs['AAPL']

In [ ]:
pd.concat([dfs['AAPL'], dfs['GOOG'], dfs['MSFT']], ignore_index=True)

In [ ]:
pd.concat(dfs.values(), ignore_index=True)

Собираем все данные за день в один `DataFrame`:

In [ ]:
def stocks_single_date(date_str):
    dfs = pd.read_excel('./stocks_data/{}.xls'.format(date_str), sheetname=None)
    for ticker, df in dfs.iteritems():
        df['Ticker'] = ticker
        df.Time = pd.to_datetime(date_str + ' ' + df.Time)
    all_tickers = pd.concat(dfs.values(), ignore_index=True)
    return all_tickers[['Time', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']]

In [ ]:
df = stocks_single_date('2016-11-25')
df

Еще один пример джойна:

In [ ]:
descriptions = pd.DataFrame.from_dict(
    {'AAPL' : {'Name' : 'Apple Inc.', 'Employees' : 116000},
     'GOOG' : {'Name' : 'Google Inc.', 'Employees' : 57100},
     'MSFT' : {'Name' : 'Microsoft Corporation', 'Employees' : 114000},
     'IBM' : {'Name' : 'IBM', 'Employees' : 380000}},
    orient='index')

In [ ]:
descriptions

In [ ]:
pd.merge(df, descriptions, how='left', left_on='Ticker', right_index=True)

Резюме:
- `pd.read_excel()` для чтения `xls`-файлов
- `pd.concat()` для объединения таблиц в простых случаях (по индексу или игнорируя индекс)
- `pd.merge()` для объединения таблиц в сложных случаях (по любым колонкам, left/right join и т.д.)
- `pd.to_datetime()` для извлечения даты и времени

## Все файлы

In [ ]:
import os

In [ ]:
os.listdir('./stocks_data/')

In [ ]:
%%time
frames = []
for filename in os.listdir('./stocks_data/'):
    if filename.endswith('.xls'):
        date_str = filename[:-4]
        df = stocks_single_date(date_str)
        frames.append(df)

In [ ]:
frames[1]

In [ ]:
df = pd.concat(frames, ignore_index=True)
df

## Работа с датами

[Документация](https://pandas.pydata.org/pandas-docs/stable/timeseries.html)

In [ ]:
aapl = df.loc[df.Ticker == 'AAPL']

In [ ]:
aapl

In [ ]:
aapl.Time.dt.date

In [ ]:
aapl.Time.dt.day

In [ ]:
aapl.Time.dt.hour

In [ ]:
aapl.Time.dt.week

In [ ]:
aapl.Time.dt.weekday

In [ ]:
aapl.Time.dt.is_quarter_end

In [ ]:
aapl.set_index('Time')

In [ ]:
aapl.set_index('Time', inplace=True)
aapl.sort_index(inplace=True)

In [ ]:
aapl[['High', 'Low']].plot()

In [ ]:
aapl.loc['2017-04':'2017-05', 'Close'].plot()

In [ ]:
aapl.loc['2017', 'Close'].plot()

In [ ]:
aapl.loc['2017-05-01':'2017-05-07', 'Close'].plot()

In [ ]:
aapl.Close.resample('1D').mean().plot()

In [ ]:
aapl.Close.resample('1W').mean().plot()

In [ ]:
aapl.Close.resample('1W').last().plot()

In [ ]:
aapl.Close.plot()
aapl.Close.shift().plot()

In [ ]:
aapl.Close.plot()
aapl.Close.shift(24).plot()

In [ ]:
aapl.Close.plot()
aapl.Close.shift(3, freq='D').plot()

In [ ]:
aapl.Close.plot()
aapl.Close.shift(5, freq=pd.offsets.BDay()).plot()

In [ ]:
(aapl.Close / aapl.Close.shift(1, freq=pd.offsets.BDay()) - 1.0).plot()

In [ ]:
aapl.Close.resample('1D').last().pct_change().plot()

In [ ]:
aapl.Close.plot()
aapl.Close.resample('1D').last().rolling(10, min_periods=1, center=True).mean().plot()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
aapl.Close.plot(label='raw')

window = aapl.Close.resample('1D').last().rolling(20, min_periods=1, center=True)
window.mean().plot(label='average')

error = 3 * window.std() / np.sqrt(window.count())
plt.fill_between(error.index, window.mean() - error, window.mean() + error,
                 alpha=0.4, color='skyblue', label='error')

plt.legend(loc='best')